<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/NN/Lab%201/Lab_1_Small_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Αϊδίνης Γιώργος 03116031

Κολιός Παναγιώτης 03116100

---

Ομάδα M.B.8

---


#Lab 1: Επιβλεπόμενη Μάθηση - Ταξινόμηση - Μικρό Dataset (S11 - Quality Assessment of Digital Colposcopies)

Αρχικά ενημερώνουμε τις βιβλιοθήκες που θα χρησιμοποιήσουμε.



In [0]:
!pip install --upgrade pip #upgrade pip package installer
!pip install scikit-learn --upgrade #upgrade scikit-learn package
!pip install numpy --upgrade #upgrade numpy package
!pip install pandas --upgrade #--upgrade #upgrade pandas package



1.   Το dataset περιγράφει μετρήσεις κολποσκοπήσεων και την κατάσταση (ετικέτα) των αντίστοιχων κόλπων, όπως προκύπτει από τις εκτιμήσεις καθενός από 6 ειδικούς και την εκτίμηση της πλειοψηφίας. Οι μετρήσεις και οι εκτιμήσεις είναι διαφορετικές για τις διαφορετικές μεθόδους με τις οποίες πραγματοποιούνται οι κολποσκοπήσεις: Hinselmann, Green, Schiller.



2.    Έχουμε συνολικά 287 δείγματα, 92 από τη μέθοδο Schiller, 98 από τη μέθοδο Green και 97 από τη μέθοδο Hinselmann. Σε κάθε δείγμα περιέχονται μετρήσεις για 62 χαρακτηριστικά-παρατηρήσεις από τα οποία προέκυψαν οι εκτιμήσεις. Όλα τα χαρακτηριστικά είναι διατεταγμένα.Οι μετρήσεις είναι αριθμητικά δεδομένα και αφορούν τιμές όπως τις επιφάνειες περιοχών του κόπλου.

3.    Υπάρχουν επικεφαλίδες στην πρώτη γραμμή πάνω από τα χαρακτηριστικά και τις ετικέτες, οι οποίες θα πρέπει να αφαιρεθούν. Δεν υπάρχει στήλη για την αρίθμηση των γραμμών.

4.    Η τιμή της κατάστασης μπορεί να πάρει δύο τιμές, 0 για κακή και 1 για καλή. Όπως υποδεικνύεται από τις FAQ το πρόβλημα θα αναλυθεί ως binary classification λαμβάνοντας υπόψην μόνο τις εκτιμήσεις της πλειοψηφίας. Έτσι οι στήλες των ετικετών που αφορούν μεμονωμένα τον κάθε ειδικό αφαιρούνται και μένει μόνο η τελευταία στήλη με τις εκτιμήσεις της πλειοψηφίας. Η στήλη αυτή είναι η τελευταία (θέση 69 στον αρχικό πίνακα). Παρατηρούμε οτι όλες οι ισοψηφίες (3-3) επιλύονται θεωρώντας την κατάσταση ως καλή, γεγονός που ίσως μας αναγκάσει να αυξήσουμε αργότερα το πλήθος των δειγμάτων με ετικέτα 0 ή να διαγράψουμε δείγματα με ετικέτα 1.

    
5.    Συνενώνουμε τα αρχεία που αφορούν τις 3 διαφορετικές μεθόδους. Μετά τη συνένωση οι επικεφαλίδες προστέθηκαν ως γραμμές, με αποτέλεσμα να έχουμε 3 φορές την ίδια γραμμή με τις επικεφαλίδες, τις οποίες και αφαιρούμε.

6.    Δεν υπάρχουν απουσιάζουσες τιμές.



In [0]:
from urllib.request import urlretrieve
import os
import zipfile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")


In [3]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/00384/Quality%20Assessment%20-%20Digital%20Colposcopy.zip","QADC.zip")
print("All the files are downloaded")

Download file... QADC.zip ...
File downloaded
All the files are downloaded


In [4]:
!ls

QADC.zip  sample_data


In [0]:
!unzip -q "QADC.zip"

In [6]:
!ls

 QADC.zip  'Quality Assessment - Digital Colposcopy'   sample_data


Rename directory for better handling



In [0]:
!mv Quality\ Assessment\ -\ Digital\ Colposcopy QADC

Remove attributes line for the 2nd and 3rd files:

In [0]:
with open("QADC/hinselmann.csv",'r') as f:
    with open("QADC/hinselmann1.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

In [0]:
with open("QADC/schiller.csv",'r') as f:
    with open("QADC/schiller1.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

create one big .csv

In [0]:
!cat QADC/green.csv QADC/hinselmann1.csv QADC/schiller1.csv > all.csv

We needed to concatenate the 3 .csv files && remove the category names from the second and third files gia na min exoume asxeta instances
We also need to remove the attribute of the experts, apart from the concensus

In [0]:
import pandas as pd
import numpy as np

df = pd.read_csv("all.csv")
df = df.drop(df.columns[[62, 63, 64, 65, 66, 67]], axis=1)

Let's print number, names and types of attributes

In [12]:
print ("We have ",len(df.columns), " attributes.")
for i in range(0, len(df.columns)):
    print('{:<10}{:<40}{:<10}{:<20}'.format(str(i+1), str(df.columns[i]),"type: ", str(df.dtypes[df.columns[i]])))


We have  63  attributes.
1         cervix_area                             type:     float64             
2         os_area                                 type:     float64             
3         walls_area                              type:     float64             
4         speculum_area                           type:     float64             
5         artifacts_area                          type:     float64             
6         cervix_artifacts_area                   type:     float64             
7         os_artifacts_area                       type:     float64             
8         walls_artifacts_area                    type:     float64             
9         speculum_artifacts_area                 type:     float64             
10        cervix_specularities_area               type:     float64             
11        os_specularities_area                   type:     float64             
12        walls_specularities_area                type:     float64             
13 

We need to count the occurences of each label (healthy or unhealthy):

In [13]:
labels_df = df.iloc[:, [62]]
labels = labels_df.values.reshape(287,)
positive = 0
negative = 0
for i in range(0, len(labels)):
    positive += labels[i]==1
    negative += labels[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  216  negative:  71


Βλεπουμε οτι εχουμε unbalanced data set, καθως η κλαση των positive(υγειων) ειναι κατα πολυ μεγαλυτερη σε μεγεθος απο την κλαση των negative. (75-25)

Train - Test split

In [15]:
from sklearn.model_selection import train_test_split


train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.2)
print(len(train))


229


In [16]:
from scipy import stats as st


# standardization
std_x = st.zscore(train)
Xvar = np.var(std_x, axis=0)
print(Xvar)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Undersampling - Oversampling

In [17]:
positive = 0
negative = 0
for i in range(0, len(train_labels)):
    positive += train_labels[i]==1
    negative += train_labels[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  174  negative:  55


In [18]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(sampling_strategy=0.65)
train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [19]:
positive = 0
negative = 0
for i in range(0, len(train_labels_oversampled)):
    positive += train_labels_oversampled[i]==1
    negative += train_labels_oversampled[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  174  negative:  113


In [20]:
from imblearn.under_sampling import RandomUnderSampler

ros = RandomUnderSampler(random_state=0)
train_undersampled, train_labels_undersampled = ros.fit_sample(train_oversampled,train_labels_oversampled)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [21]:
positive = 0
negative = 0
for i in range(0, len(train_labels_undersampled)):
    positive += train_labels_undersampled[i]==1
    negative += train_labels_undersampled[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  113  negative:  113


In [22]:
Xvar = np.var(train_undersampled, axis=0)
print(Xvar)

[5.92466131e-02 4.31792222e-05 5.01420473e-02 3.06785137e-02
 3.16436337e-03 2.73114680e-03 1.68542817e-02 9.17460067e-03
 8.93209624e-03 5.01644887e-04 8.31705790e-04 1.29625372e-03
 1.39650379e-02 1.66864628e-03 2.09571220e-02 2.23180364e+03
 2.63681993e+02 1.46290470e+03 3.52806657e+03 2.67293229e+03
 2.76319942e+02 1.75927367e+03 4.13923081e+03 4.07913972e+03
 5.00045348e+02 2.47323610e+03 6.68513404e+03 1.21361214e+03
 4.89639824e+02 5.32753460e+02 2.87375046e+03 1.22319994e+03
 4.63961935e+02 6.04363787e+02 2.76995996e+03 2.53844926e+03
 6.21760686e+02 1.08594467e+03 5.23447522e+03 5.37100728e-01
 3.69373855e-01 3.19553909e+03 3.64079486e+02 3.79352345e+03
 4.27196549e+02 4.45141380e-01 5.57563729e-02 2.89751787e+03
 3.70168504e+02 1.77118042e+03 4.67106971e+02 3.66858127e-02
 5.77056007e-02 3.46212996e-02 6.30513543e-02 1.98587036e-02
 1.94964821e-01 3.93067712e-02 5.25935905e-02 9.14256264e+03
 6.99555879e-02 3.13637317e-02 2.50000000e-01]


In [23]:
from scipy import stats as st
from sklearn import preprocessing


train_scaled = preprocessing.scale(train_undersampled)

Xvar = np.var(train_scaled, axis=0)
print(Xvar)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [24]:
Yvar = np.var(np.asarray(test), axis=0)
print(Yvar)

[4.13075562e-02 3.33472336e-05 3.57819903e-02 3.59033170e-02
 8.12577089e-03 6.33184412e-03 3.88708020e-02 1.03793711e-02
 1.61867476e-02 1.34534109e-03 5.66579103e-03 4.69626975e-04
 2.07500710e-02 1.46236523e-03 2.00129931e-02 2.76160963e+03
 3.15053750e+02 1.87173681e+03 4.28158996e+03 2.70171816e+03
 2.79684762e+02 1.79216655e+03 4.17063928e+03 4.31318092e+03
 4.44655244e+02 2.76999869e+03 6.74567363e+03 1.23593850e+03
 3.58278154e+02 7.07822354e+02 2.48061095e+03 1.23411367e+03
 3.63617353e+02 8.07946307e+02 2.38751573e+03 2.12574925e+03
 4.43553424e+02 1.07113247e+03 4.06747288e+03 5.03902759e-01
 4.41555549e-01 3.28901851e+03 5.42474681e+02 3.67403051e+03
 3.45962773e+02 4.22216638e-01 5.61482090e-02 3.00422001e+03
 3.58902384e+02 1.21698788e+03 2.56355404e+02 4.48002160e-02
 4.23182341e-02 3.69514507e-02 5.08940465e-02 2.24262072e-02
 1.35868266e-01 4.71380410e-02 3.88613991e-02 7.14658470e+03
 7.00329978e-02 3.23766685e-02 1.99762188e-01]


In [25]:
scaler = preprocessing.StandardScaler().fit(train_undersampled)
test_scaled = scaler.transform(test)
Yvar = np.var(np.asarray(test_scaled), axis=0)
print(Yvar)

[0.6972138  0.77229815 0.71361247 1.17030823 2.56790069 2.31838293
 2.30628648 1.13131584 1.81220031 2.68185947 6.81225393 0.36229557
 1.48585855 0.8763782  0.9549495  1.23738916 1.19482467 1.27946599
 1.2135797  1.01076939 1.01217726 1.01869686 1.007588   1.05737514
 0.88922984 1.11998959 1.00905585 1.01839662 0.73171776 1.32861146
 0.86319636 1.00892227 0.78372238 1.33685427 0.86193149 0.83742042
 0.71338287 0.98636008 0.77705457 0.93819042 1.19541636 1.0292531
 1.48998969 0.9685008  0.80984449 0.94850009 1.00702765 1.03682536
 0.96956489 0.68710554 0.5488152  1.22118641 0.73334709 1.06730397
 0.80718403 1.12928858 0.69688606 1.19923463 0.73889991 0.78168288
 1.00110656 1.03229644 0.79904875]


In [0]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from scipy import stats as st
from sklearn import neighbors
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier


"""
    This is a function that applies a transformation (selection) on a dataset, 
    based on the type of the selector. Takes 3 arguments, one is mandatory and 
    is the dataset, which is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The other two can be
    None, and when they are, no transformation happens. The selector argument
    is a string and can be one of two things: either "PCA" or 
    "Variance Threshold". In either case, there can be arguments, and in "PCA"
    the arguments can be an int of the number of principal components, and in
    "Variance Threshold", it can be a float. Returns the transformed dataset.
"""
def Apply_selector(data, selector = None, arguments = None):
    (train, train_labels, test, test_labels) = data
    if selector == "PCA":    
        if arguments:
            number_of_components = arguments
            pca = PCA(n_components = number_of_components)
            pca.fit(train)
            train_reduced = pca.transform(train)
            test_reduced = pca.transform(test)
        else:
            pca = PCA()
            pca.fit(train)
            train_reduced = pca.transform(train)
            test_reduced = pca.transform(test)
    elif selector == "Variance Threshold":
        if arguments:
            t = arguments
            sel = VarianceThreshold(threshold=t)
            train_reduced = sel.fit_transform(train)
            test_reduced = sel.transform(test)
        else:
            sel = VarianceThreshold()
            train_reduced = sel.fit_transform(train)
            test_reduced = sel.transform(test)
    else:
        (train_reduced, train_labels, test_reduced, test_labels) = data
    return (train_reduced, train_labels, test_reduced, test_labels)


"""
    This is a function that applies resampling on a dataset, based on the 
    type of sampling method. Takes 3 arguments, one is mandatory and 
    is the dataset, which is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The other two can be
    None, and when they are, no sampling happens. The sampling argument
    is a string and can be one of three things: "Over", "Under" or 
    "OverUnder". In any case, there can be arguments, and in Under or Over or
    OverUnder it can be a float with the ratio of samples between classes. 
    Returns the resampled dataset.
"""
def Apply_sampling(data, sampling = None, arguments = None):
    (train, train_labels, test, test_labels) = data
    if sampling == "Over":    
        if arguments:
            ratio = arguments
            ros = RandomOverSampler(sampling_strategy=ratio)
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
        else:
            ros = RandomOverSampler()
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
        return (train_oversampled, train_labels_oversampled, test, test_labels)
    elif sampling == "Under":
        if arguments:
            ratio = arguments
            rus = RandomUnderSampler(sampling_strategy=ratio)
            train_undersampled, train_labels_undersampled = rus.fit_sample(train,train_labels)
        else:
            rus = RandomUnderSampler()
            train_undersampled, train_labels_undersampled = rus.fit_sample(train,train_labels)
        return (train_undersampled, train_labels_undersampled, test, test_labels)
    elif sampling=="OverUnder":
        if arguments:    
            ratio = arguments
            ros = RandomOverSampler(sampling_strategy=ratio)
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
            rus = RandomUnderSampler()
            train_undersampled, train_labels_undersampled = rus.fit_sample(train_oversampled,train_labels_oversampled)
        else:
            ros = RandomOverSampler()
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
            rus = RandomUnderSampler()
            train_undersampled, train_labels_undersampled = rus.fit_sample(train_oversampled,train_labels_oversampled)
        return (train_undersampled, train_labels_undersampled, test, test_labels)
    else:
        return data



"""
    This is a function that applies a transformation (standardization) on a dataset, 
    based on the type of the standardizer. Takes 3 arguments, one is mandatory and 
    is the dataset, which is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The other two can be
    None, and when they are, no transformation happens. The selector argument
    is a string and can be one of two things: either "zscore" or 
    "minmax". Argument arguments is always None. Returns the transformed dataset.
"""
def Apply_standardizer(data, standardizer = None,  arguments = None):
    (train, train_labels, test, test_labels) = data
    if standardizer=="zscore":
        std_train = st.zscore(train)
    elif standardizer=="minmax":
        std_train = (train - np.min(train) )/ (np.max(train) - np.min(train))
    else:
        std_train = train
    return (std_train, train_labels, test, test_labels)

"""
    This is a function that creates a classifier object. It does not train, 
    it takes the name of the classifier, along with the arguments we want 
    to pass to the instance creator for the classifier and returns the 
    classifier object.
"""
def Create_classifier(classifier,  arguments = None):
    if classifier=="kNN":
        if arguments:
            neighbors = arguments
            clf = neighbors.KNeighborsClassifier(n_jobs = -1, n_neighbors = neighbors)
        else:
            clf = neighbors.KNeighborsClassifier(n_jobs = -1)
    elif classifier=="dummy":
        if arguments:
            strat = arguments
            clf = DummyClassifier(strategy=strat)
        else:
            clf = neighbors.KNeighborsClassifier()
    return clf


In [0]:
"""
    This is an implementation for the function imblearn.pipeline.Pipeline().
    The way this function works is, it takes as input two arguments. The first
    is a list. This list contains the steps that need to be executed in the 
    pipeline, IN ORDER. Always the last step is the classifier. Each step is
    a tuple, which contains the name of the step, and the arguments needed to 
    execute the step, like so: ("name", arguments). The second argument of the 
    function is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The function outputs a
    tuple, which contains the processed data, ready for fitting the classier, 
    and the classifier object, as specified by the arguments, like so:
    returns: (processed_data, classifier_object)
"""
def Pipeline(steps, data):
    steps_dict = {"selector": Apply_selector,
                  "sampling": Apply_sampling,
                  "standardizer": Apply_standardizer,
                  "classifier": Create_classifier}
    for step in steps:
        if step[0]!="classifier":
            data = steps_dict[step[0]](data,steps[1])
        else:
            return ( data, steps_dict[step[0]](steps[1]) )


"""
    This is an implementation for the function gridsearchcv.blabla u know
"""
def model_tuning(model, data, parameters = None):
    pass
         

In [73]:
df = pd.read_csv("all.csv")
df = df.drop(df.columns[[62, 63, 64, 65, 66, 67]], axis=1)
train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.2)
print(len(train), len(test))
data = (train, train_labels, test,  test_labels)

data, clf = Pipeline([("selector", "PCA"),("sampling","Under",0.5), ("standardizer", "zscore"), ("classifier","kNN")], data)


229 58


In [0]:

parameters = "Uniform"
model_tuning("dummy", data, parameters)

parameters = [1,2,3,4,4,5,6,7,8,9,10]
model_tuning("kNN", data, parameters)
